#### Import required libaries and data

In [1]:
# used for dataframe manipulation
import pandas as pd

# linear programming modeler written in Python
from pulp import *

# used for various mathematical operations
import numpy as np

warnings.filterwarnings("ignore")

In [2]:
# Import product for weight estimation
df = pd.read_excel('example.xlsx')
df

,Ingredient number,Ingredient,Ingredient weight (g/100g),Subingredient number,Subingredient,Subingredient weight (g/100g),Leaf ingredient number,Leaf ingredient,Leaf ingredient weight (g/100g),Sugar,Starch,Protein,Sat fat,Unsat fat,Sodium
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2.0,2.0,2.00,3.00,3
1,01,"chia seed gel (40%) (filtered water , chia seed)",40.0,01,filtered water,NaN,0101,filtered water,NaN,0.0,0.0,0.0,0.00,0.00,1
2,01,"chia seed gel (40%) (filtered water , chia seed)",40.0,02,chia seed,NaN,0102,chia seed,NaN,0.0,7.7,16.5,3.34,27.36,16
3,02,apple puree (35%),35.0,NaN,NaN,NaN,02,apple puree,35.0,11.9,0.8,0.3,0.00,0.20,1
4,03,coconut milk,NaN,NaN,NaN,NaN,03,coconut milk,NaN,1.5,0.2,1.3,11.50,1.30,18
5,04,apple dice (8%),8.0,NaN,NaN,NaN,04,apple dice,8.0,11.9,0.8,0.3,0.00,0.20,1
6,05,organic cane sugar,NaN,NaN,NaN,NaN,05,organic cane sugar,NaN,100.0,0.0,0.0,0.00,0.00,0
7,06,lemon juice concentrate,NaN,NaN,NaN,NaN,06,lemon juice concentrate,NaN,39.6,0.0,0.2,0.00,0.00,37
8,07,cinnamon (0.4%),0.4,NaN,NaN,NaN,07,cinnamon,0.4,13.8,11.7,4.2,1.35,1.35,18


#### Initiate the LP problem and construct a list of main ingredient weights

In [3]:
# Initiate linear programming problem, specifying minimization
prob = LpProblem("Ingredient weight prediction algorithm", LpMinimize)

# Create an LP variable for each main ingredient weight, specifying a lower bound of 0.01g per 100g (Constraint 1a)
ingredient_numbers = df['Ingredient number'].dropna().unique().tolist()
ingredient_weights = [LpVariable('weight of ingredient ' + str(x), 0.01, None) for x in ingredient_numbers]
ingredient_weights

[weight_of_ingredient_01,
 weight_of_ingredient_02,
 weight_of_ingredient_03,
 weight_of_ingredient_04,
 weight_of_ingredient_05,
 weight_of_ingredient_06,
 weight_of_ingredient_07]

In [4]:
# Incorporate any known main ingredient weights (Constraint 2a)
known_ingredient_weights = df.drop_duplicates(subset = 'Ingredient number')['Ingredient weight (g/100g)'][1:].tolist()
for i in range(0, len(ingredient_weights)):
    if known_ingredient_weights[i]>0:
        ingredient_weights[i] = known_ingredient_weights[i]
ingredient_weights

[40.0,
 35.0,
 weight_of_ingredient_03,
 8.0,
 weight_of_ingredient_05,
 weight_of_ingredient_06,
 0.4]

#### Introduce main ingredient labelling constraints

In [5]:
# Main ingredient weights must sum to 100g per 100g (Constraint 3)
prob += sum(ingredient_weights) == 100

# Main ingredient weights must be in descending weight order (Constraint 4)
for i in range(0, len(ingredient_weights) - 1):
    prob += ingredient_weights[i] >= ingredient_weights[i+1]

#### Extend the above constraints to products that contain subingredients

In [6]:
# Find the numbers of all compound ingredients 
compound_ingredient_numbers = df[df['Ingredient number'].duplicated()]['Ingredient number'].unique().tolist()
compound_ingredient_numbers

['01']

In [7]:
# create an empty list. Append (compound ingredient number, subingredient weights) tuples to this later on
compound_ingredient_info = []

# Loop through each compound ingredient number
for number in compound_ingredient_numbers:
    
    # Access the compound ingredient
    index = int(float(number))-1
    compound_ingredient = ingredient_weights[index]
    
    # Create an LP variable for each subingredient weight in the compound ingredient, specifying a lower bound of 0.01
    # (CONSTRAINT 1B)
    subingredient_numbers = df[df['Ingredient number'] == number]['Subingredient number'].tolist()
    subingredient_weights = [LpVariable('weight of ingredient ' + number + x, 0.01, None) for x in subingredient_numbers]

    # Insert any known subingredient weights (CONSTRAINT 2A)
    known_subingredient_weights = df[df['Ingredient number'] == number]['Leaf ingredient weight (g/100g)'].tolist()
    for i in range(0, len(subingredient_weights)):
        if known_subingredient_weights[i]>0:
            subingredient_weights[i] = known_subingredient_weights[i]
    
    # The weight of the compound ingredient is equal to the sum of the subingredient weights (CONSTRAINT 5)
    prob += compound_ingredient == sum(subingredient_weights)
    
    # Subingredient weights must be in descending order (CONSTRAINT 6)
    for j in range(0, len(subingredient_weights) - 1):
        prob += subingredient_weights[j] >= subingredient_weights[j+1]
        
    # Append compound ingredient number and subingredient weights to list above
    compound_ingredient_info.append((number, subingredient_weights))

#### Construct a list of leaf ingredient weights

In [8]:
# Wherever a main ingredient weight is composed of subingredients, 
# override the the main ingredient weight with the list of subingredient weights
for info in compound_ingredient_info:
    compound_ingredient_index = int(float(info[0]))-1
    compound_ingredient_subingredient_weights = info[1]
    ingredient_weights[compound_ingredient_index] = compound_ingredient_subingredient_weights

# Now construct a flat list of all the "leaf" ingredient weights
leaf_ingredient_weights = []
for weight in ingredient_weights:
    if type(weight) == list:
        for weight2 in weight:
            leaf_ingredient_weights.append(weight2)
    else:
        leaf_ingredient_weights.append(weight)
        
leaf_ingredient_weights

[weight_of_ingredient_0101,
 weight_of_ingredient_0102,
 35.0,
 weight_of_ingredient_03,
 8.0,
 weight_of_ingredient_05,
 weight_of_ingredient_06,
 0.4]

#### Construct the Objective Function

In [9]:
# Calculated values are determined using the dot product of leaf ingredient weights and leaf ingredient nutrients
difference_sugar = 100*df['Sugar'][0] - np.array(leaf_ingredient_weights).dot(df[['Sugar']].as_matrix()[1:])[0]
difference_starch = 100*df['Starch'][0] - np.array(leaf_ingredient_weights).dot(df[['Starch']].as_matrix()[1:])[0]
difference_protein = 100*df['Protein'][0] - np.array(leaf_ingredient_weights).dot(df[['Protein']].as_matrix()[1:])[0]
difference_satfat = 100*df['Sat fat'][0] - np.array(leaf_ingredient_weights).dot(df[['Sat fat']].as_matrix()[1:])[0]
difference_unsatfat = 100*df['Unsat fat'][0] - np.array(leaf_ingredient_weights).dot(df[['Unsat fat']].as_matrix()[1:])[0]
difference_sodium = 100*df['Sodium'][0] - np.array(leaf_ingredient_weights).dot(df[['Sodium']].as_matrix()[1:])[0]

In [10]:
# Construct auxiliary variables
sugar_aux = LpVariable("sugar_aux", 0, None)
starch_aux = LpVariable("starch_aux", 0, None)
protein_aux = LpVariable("protein_aux", 0, None)
satfat_aux = LpVariable("satfat_aux", 0, None)
unsatfat_aux = LpVariable("unsatfat_aux", 0, None)
sodium_aux = LpVariable("sodium_aux", 0, None)

# set up the objective function. Divide auxiliaries by standard deviations
prob += sugar_aux*(1/20) + starch_aux*(1/21) + protein_aux*(1/10) + satfat_aux*(1/8) + unsatfat_aux*(1/13) + sodium_aux*(1/1600)

# Set additional constraints for auxiliary variables. 
prob += sugar_aux >= difference_sugar
prob += sugar_aux >= -difference_sugar
prob += starch_aux >= difference_starch
prob += starch_aux >= -difference_starch
prob += protein_aux >= difference_protein
prob += protein_aux >= -difference_protein
prob += satfat_aux >= difference_satfat
prob += satfat_aux >= -difference_satfat
prob += unsatfat_aux >= difference_unsatfat
prob += unsatfat_aux >= -difference_unsatfat
prob += sodium_aux >= difference_sodium
prob += sodium_aux >= -difference_sodium

#### Solve the problem

In [11]:
prob

Ingredient_weight_prediction_algorithm:
MINIMIZE
0.1*protein_aux + 0.125*satfat_aux + 0.000625*sodium_aux + 0.047619047619047616*starch_aux + 0.05*sugar_aux + 0.07692307692307693*unsatfat_aux + 0.0
SUBJECT TO
_C1: weight_of_ingredient_03 + weight_of_ingredient_05
 + weight_of_ingredient_06 = 16.6

_C2: weight_of_ingredient_03 <= 35

_C3: weight_of_ingredient_03 >= 8

_C4: weight_of_ingredient_05 <= 8

_C5: weight_of_ingredient_05 - weight_of_ingredient_06 >= 0

_C6: weight_of_ingredient_06 >= 0.4

_C7: weight_of_ingredient_0101 + weight_of_ingredient_0102 = 40

_C8: weight_of_ingredient_0101 - weight_of_ingredient_0102 >= 0

_C9: sugar_aux + 1.5 weight_of_ingredient_03 + 100 weight_of_ingredient_05
 + 39.6 weight_of_ingredient_06 >= 382.78

_C10: sugar_aux - 1.5 weight_of_ingredient_03 - 100 weight_of_ingredient_05
 - 39.6 weight_of_ingredient_06 >= -382.78

_C11: starch_aux + 7.7 weight_of_ingredient_0102 + 0.2 weight_of_ingredient_03
 >= 160.92

_C12: starch_aux - 7.7 weight_of_ingre

In [12]:
prob.solve()

1

In [13]:
# create new column for predicted weights
all_weights = []
all_weights.append(np.NaN)
for weight in leaf_ingredient_weights:
    if type(weight) ==float:
        all_weights.append(weight)
    elif type(weight) == pulp.LpVariable:
        all_weights.append(weight.varValue)
        
df['Predicted weight (g/100g)'] = np.round(all_weights, 1)
df

,Ingredient number,Ingredient,Ingredient weight (g/100g),Subingredient number,Subingredient,Subingredient weight (g/100g),Leaf ingredient number,Leaf ingredient,Leaf ingredient weight (g/100g),Sugar,Starch,Protein,Sat fat,Unsat fat,Sodium,Predicted weight (g/100g)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2.0,2.0,2.00,3.00,3,NaN
1,01,"chia seed gel (40%) (filtered water , chia seed)",40.0,01,filtered water,NaN,0101,filtered water,NaN,0.0,0.0,0.0,0.00,0.00,1,29.8
2,01,"chia seed gel (40%) (filtered water , chia seed)",40.0,02,chia seed,NaN,0102,chia seed,NaN,0.0,7.7,16.5,3.34,27.36,16,10.2
3,02,apple puree (35%),35.0,NaN,NaN,NaN,02,apple puree,35.0,11.9,0.8,0.3,0.00,0.20,1,35.0
4,03,coconut milk,NaN,NaN,NaN,NaN,03,coconut milk,NaN,1.5,0.2,1.3,11.50,1.30,18,12.7
5,04,apple dice (8%),8.0,NaN,NaN,NaN,04,apple dice,8.0,11.9,0.8,0.3,0.00,0.20,1,8.0
6,05,organic cane sugar,NaN,NaN,NaN,NaN,05,organic cane sugar,NaN,100.0,0.0,0.0,0.00,0.00,0,3.5
7,06,lemon juice concentrate,NaN,NaN,NaN,NaN,06,lemon juice concentrate,NaN,39.6,0.0,0.2,0.00,0.00,37,0.4
8,07,cinnamon (0.4%),0.4,NaN,NaN,NaN,07,cinnamon,0.4,13.8,11.7,4.2,1.35,1.35,18,0.4
